<a href="https://colab.research.google.com/github/HoseinNekouei/fine_tuning_with_custom_dataset/blob/main/fine_tuning_imdb_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Dataset**

In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2025-02-25 08:10:01--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  4.50MB/s    in 31s     

2025-02-25 08:10:33 (2.55 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



# **Load the dataset and divide into training and testing set**

In [2]:
from posixpath import join
from pathlib import Path

def read_imdb_split(split_dir):
  split_dir= Path(split_dir)
  texts=[]
  labels=[]

  for label_dir in ['neg', 'pos']:
    for text_file in (split_dir/label_dir).iterdir():

      texts.append(text_file.read_text())
      labels.append(0 if label_dir== 'neg' else 1)

  return texts, labels

train_texts, train_labels= read_imdb_split('/content/aclImdb/train')
# test_texts, test_labels= read_imdb_split('/content/aclImdb/test')

shape=','.join(
    [f'train text length: {len(train_texts)}',
    f'train labels length: {len(train_labels)}',
    # f'test text length: {len(test_texts)}',
    # f'test labels length: {len(test_labels)}'
    ])

print(shape)

train text length: 25000,train labels length: 25000


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_texts, train_labels, test_size= 0.2, stratify= train_labels)

length= ','. join([
    f'X_train length: {len(X_train)}',
    f'X_val length: {len(X_val)}',
    f'y_train length: {len(y_train)}',
    f'y_val length: {len(y_val)}'
])

print(length)

X_train length: 20000,X_val length: 5000,y_train length: 20000,y_val length: 5000


# **Load Tokenizer**

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased-finetuned-sst-2-english')
tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length= 256)
val_encodings= tokenizer(X_val, padding= True, truncation= True, max_length= 256)
# test_encodings= tokenizer(test_texts, padding= True, truncation= True, max_length= 256)

# **Make Dataset and DataLoader**

In [6]:
import torch

In [ ]:
class IMDBDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings= encodings
    self.labels= labels

  def __getitem__(self, idx):
    item= {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


train_set= IMDBDataset(train_encodings, y_train)
val_set= IMDBDataset(val_encodings, y_val)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = DataLoader(val_set, batch_size=128)

In [ ]:
from transformers import  AutoModelForSequenceClassification
from torch.optim import AdamW, SGD, Adam
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

checkpoint= 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.to(device)

# optim = AdamW(model.parameters(), lr=5e-5)
optim = Adam(model.parameters(), lr=5e-5)
# optim = SGD(model.parameters(), lr=0.001,momentum =0.9)

In [10]:
# batch =next(iter(train_loader))
# input_ids= batch['input_ids'].to(device)
# attention_mask = batch['attention_mask'].to(device)
# labels= batch['labels'].to(device)

# outputs= model(input_ids, attention_mask=attention_mask, labels=labels)
# print(outputs.logits)
# y_pred = torch.argmax(outputs.logits, dim=1)

# print('y_pred',y_pred)
# print('labels',labels)

# acc= y_pred == labels
# result= torch.sum(y_pred==labels).item()
# print(acc)
# print(result)

In [16]:
import math
epochs= 3
best_loss= float('inf')

for epoch in range(epochs):
  total_loss, train_loss, val_loss = 0, 0, 0
  total_acc, train_acc, val_acc = 0, 0 , 0

  # Set model to training mode
  model.train()

  for index, batch in enumerate(train_loader):

    # Using GPU
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # Forward pass
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    y_pred = torch.argmax(outputs.logits, dim=1)

    # Compute loss
    loss = outputs[0]

    # # Backward pass and optimization
    # loss.backward()

    # # update
    # optim.step()
    # optim.zero_grad()

    # Accumulate loss and accuracy
    train_loss += loss.item() * len(batch['input_ids'])
    train_acc += torch.sum(y_pred == labels).item()

    if (index +1) % 10 == 0:
      print(f' batch: [{index + 1}/{math.floor(len(train_set)/128)}]')

  # Average loss and accuracy for training
  train_loss /= len(train_set)
  train_acc /= len(train_set)

  # Set model to evaluation mode
  model.eval()

  with torch.no_grad():
    for batch in val_loader:

      # Using GPU
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      # Forward pass
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      y_pred= torch.argmax(outputs.logits, dim=1)

      # Compuute loss
      loss= outputs[0]

      # Accumulate loss and accuracy
      val_loss += loss.item() * len(batch['input_ids'])
      val_acc += torch.sum(y_pred == labels).item()

    # Average loss and accuracy for validation
    val_loss /= len(val_set)
    val_acc /= len(val_set)

  # Print training and validation results
  training_result= ','.join([
    f'Epoch:[{epoch + 1}], ',
    f'train_Loss:{train_loss:.3f}, ',
    f'train_acc:{train_acc:.3f}, '
    f'val_Loss:{val_loss:.3f}, ',
    f'val_acc:{val_acc:.3f}'
  ])

  print(training_result)

  # Save the model if validation loss improves
  if val_loss < best_loss:
    best_loss= val_loss
    print('model saved!')
    torch.save(model, 'imdb_model.pt')


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# with torch.no_grad():
#     outputs = model(input_ids, attention_mask=attention_mask)
#     predictions = torch.argmax(outputs.logits, dim=-1)